##Installation of the Libraries/Packages

In [ ]:
!pip install stanza

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
!pip install rouge

##Pre-Processing

In [ ]:
!pdftotext -layout /content/2022-uosc-securityandfirelreport-1001bcleryreport.pdf /content/2022-uosc-securityandfirelreport-1001bcleryreport.txt

In [2]:
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        return f.readlines()

In [ ]:
#Removed Footer information outside by the Regular Expressions and created chapter wise txt files.
#Resolving Line break Issue
import os
  
# Folder Path 
path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/"
  
# Change the directory
os.chdir(path)
New_Lines = []
for file in os.listdir():
    New_Lines = []
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}{file}"
  
        # call read text file function
        Lines = read_text_file(file_path)
        new_line = "";
        for line in Lines:
          if(line.strip() != ""):
            new_line = new_line +" "+ line.strip()
            continue;
          if(new_line != ""):
            New_Lines.append(new_line)
          new_line = ""
        if(new_line != ""):
            New_Lines.append(new_line)
        with open(r'/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/'+file, 'w') as fp:
          fp.write('\n'.join(New_Lines))  
    print(len(New_Lines))
print(len(New_Lines)) 

##Sentiment


In [3]:
def listtostring(list):
  document = ""
  for line in list:
    document += line+" "
  return document

###Tradtional Method

In [ ]:
#Sentiment Traditional NLP
#Ref - https://nlp.stanford.edu/pubs/qi2020stanza.pdf
#0 - Negative, 1 - Neutral, 2 - Possitive

import stanza
import pandas as pd
import os

nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment', tokenize_no_ssplit=True)

path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/"
df = pd.DataFrame()

fileNameL = []
sentimentFL = []
positive = 0
negative = 0
neutral = 0
os.chdir(path)
for file in os.listdir():
  if file.endswith(".txt"):
    file_path = f"{path}{file}"
    #print(file)
    fileName = file.replace('.txt','')
    fileNameL.append(fileName)
    Lines = read_text_file(file_path)
    doc = nlp(Lines)
    sentimentL = []
    for i, sentence in enumerate(doc.sentences):
      label = ""
      if sentence.sentiment == 0:
        label = 'Negative'
        negative += 1
      elif sentence.sentiment == 2:
        label = 'Positive'
        positive += 1
      else:
        label = 'Neutral'
        neutral += 1
      #print(i, label)
      sentimentL.append(label)
    tdf = pd.DataFrame()
    tdf[fileName] = sentimentL
    df = pd.concat([df,tdf],axis=1)
    #sentimentFL.append(sentimentL)
    #print(sentimentL)
df

In [ ]:
df.to_csv('/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/Sentiment/sentiment.csv')

###LLM based Method

In [8]:
#Sentiment LLM Based Approach, https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis?text=I+like+you.+I+love+you
from transformers import pipeline
import stanza
import pandas as pd
import os

path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/"

df = pd.DataFrame()
sentiment = pipeline('sentiment-analysis', model = 'finiteautomata/bertweet-base-sentiment-analysis')

fileNameL = []
sentimentFL = []
os.chdir(path)
for file in os.listdir():
  if file.endswith(".txt"):
    file_path = f"{path}{file}"
    print(file)
    fileName = file.replace('.txt','')
    fileNameL.append(fileName)
    Lines = read_text_file(file_path)
    print(len(Lines))
    results = sentiment(Lines, truncation=True)
    sentimentL = []
    for result in results:
      #print(result['label'])
      label = ""
      if result['label'] == 'NEG':
        label = 'Negative'
      elif result['label'] == 'POS':
        label = 'Positive'
      else:
        label = 'Neutral'
      #print(i, label)
      sentimentL.append(label)
    tdf = pd.DataFrame()
    tdf[fileName] = sentimentL
    df = pd.concat([df,tdf],axis=1)
    #sentimentFL.append(sentimentL)
    print(sentimentL)
df

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

The Division of Law Enforcement and Safety.txt
14
['Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral']
Reporting Criminal Incidents and Other Emergencies.txt
40
['Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral']
Sexual Assault, Domestic Violence, Dating Violence.txt
166
['Negative', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Negative', 'Negative', 'Negat

,The Division of Law Enforcement and Safety,Reporting Criminal Incidents and Other Emergencies,"Sexual Assault, Domestic Violence, Dating Violence",CONFIDENTIAL REPORTING OF CRIME AND OTHER SERIOUS,Crime Prevention,Monitoring and Recording Criminal Activity Includi,Missing Resident Students,Timely Warning Notices,Emergency Response,Alcohol And Drug Policies,Criminal statistics,Annual Fire Safety Report
0,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Negative,Negative,Neutral,Negative,Neutral,Neutral
1,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Neutral,Neutral
2,Positive,Negative,Neutral,Neutral,Neutral,Neutral,Negative,Neutral,Negative,Neutral,Neutral,Neutral
3,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Negative,Negative,Neutral,Neutral,Neutral,Neutral
4,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Negative,Neutral,Neutral,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
161,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.to_csv('/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/Sentiment/sentiment_llm.csv')

##Entity Extraction

###Traditional Method

In [ ]:
#Traditional Method
import os 
import stanza
import pandas as pd

path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/"
os.chdir(path)

nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
file1 = open("/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/NER/traditional_strip.csv", "w")
for file in os.listdir():
  if file.endswith(".txt"):
    jsonResult = []
    file_path = f"{path}{file}"
    print(file)
    fileName = file.replace('.txt','')
    Lines = read_text_file(file_path)
    for line in Lines:
      result = nlp(line.strip())
      for ent in result.ents:
        entity = ent.text
        entitytype = ent.type
        entitybegin = ent.start_char
        entityend = ent.end_char
        file1.write(f"{fileName}\t{entity}\t{entitytype}\t{entitybegin}\t{entityend}\n")
   # with open(r'/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/NER/'+fileName+'.json', 'w') as fp:
    #      fp.write(str(jsonResult))
file1.close()

###LLM Based Method

In [ ]:
#Method using LLM
from transformers import pipeline
nlp = pipeline('ner', model = 'dslim/bert-large-NER')
file1 = open("/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/NER/ner_llm_strip.csv", "w")
for file in os.listdir():
  if file.endswith(".txt"):
    jsonResult = []
    file_path = f"{path}{file}"
    print(file)
    fileName = file.replace('.txt','')
    #fileNameL.append(fileName)
    Lines = read_text_file(file_path)
    for line in Lines:
      results = nlp(line.strip())
      for result in results:
        entity = result['word']
        entitytype = result['entity']
        entitybegin = result['start']
        entityend = result['end']
        file1.write(f"{fileName}\t{entity}\t{entitytype}\t{entitybegin}\t{entityend}\n")
      #jsonResult.append(result)
   # with open(r'/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/NER/'+fileName+'_llm.json', 'w') as fp:
    #      fp.write(str(jsonResult))
file1.close()

In [ ]:
input_file_path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/NER/ner_llm_strip.csv"

def merge_BI_tag(input_file_path, writer):
    with open(input_file_path) as reader:
        text = ""
        beg = -1
        end = -1
        file_name = ""
        prev_end = 0
        entity_type = 'O'
        for line in reader:
            items = line.strip().split('\t')
            if items[1].startswith('##'):
                items[1] = items[1][2:]
            if items[2].startswith('B-'):
                if text != "":
                    writer.write(f"{file_name}\t{text}\t{entity_type}\t{beg}\t{end}\n")
                text = items[1]
                beg = int(items[3])
                end = int(items[4])
                file_name = items[0]
                entity_type = items[2][2:]
                prev_end = int(items[4])
            else:
                space = [' '] * (int(items[3]) - prev_end)
                text = text + ''.join(space) + items[1]
                prev_end = int(items[4])
    writer.close()

writer = open('/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/ner_llm_strip_merge_ent.tsv', 'w')
merge_BI_tag(input_file_path, writer)

##Summarization

###Traditional Method

In [4]:
from collections import Counter 
from string import punctuation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stop_words

In [5]:
def tokenizer(s):
    tokens = []
    for word in s.split(' '):
        tokens.append(word.strip().lower())
    return tokens

def sent_tokenizer(s):
    sents = []
    for sent in s.split('.'):
        sents.append(sent.strip())
    return sents

def count_words(tokens):
    word_counts = {}
    for token in tokens:
        if token not in stop_words and token not in punctuation:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

def word_freq_distribution(word_counts):
    freq_dist = {}
    max_freq = max(word_counts.values())
    for word in word_counts.keys():  
        freq_dist[word] = (word_counts[word]/max_freq)
    return freq_dist

def score_sentences(sents, freq_dist, max_len=40):
    sent_scores = {}  
    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word.lower() in freq_dist.keys():
                if len(words) < max_len:
                    if sent not in sent_scores.keys():
                        sent_scores[sent] = freq_dist[word.lower()]
                    else:
                        sent_scores[sent] += freq_dist[word.lower()]
    return sent_scores

def summarize(sent_scores, k):
    top_sents = Counter(sent_scores) 
    summary = ''
    scores = []
    
    top = top_sents.most_common(k)
    for t in top: 
        summary += t[0].strip() + '. '
        scores.append((t[1], t[0]))
    return summary[:-1], scores

In [10]:
import os
from rouge import Rouge
import math 

rouge = Rouge()
path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/"
os.chdir(path)

for file in os.listdir():
  if file.endswith(".txt"):
    file_path = f"{path}{file}"
    print(file)
    fileName = file.replace('.txt','')
    Lines = read_text_file(file_path)
    topic_content = listtostring(Lines)
    tokens = tokenizer(topic_content)
    sents = sent_tokenizer(topic_content)
    word_counts = count_words(tokens)
    freq_dist = word_freq_distribution(word_counts)
    sent_scores = score_sentences(sents, freq_dist)
    summary, summary_sent_scores = summarize(sent_scores, math.ceil(len(Lines)/10))
    print(rouge.get_scores(summary, topic_content, avg=True))
    with open(r'/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/Summarization/Traditional/'+fileName+'.txt', 'w') as fp:
      fp.write(summary)

The Division of Law Enforcement and Safety.txt
{'rouge-1': {'r': 0.12238805970149254, 'p': 1.0, 'f': 0.21808510443993326}, 'rouge-2': {'r': 0.10469314079422383, 'p': 0.9830508474576272, 'f': 0.18923327721626854}, 'rouge-l': {'r': 0.12238805970149254, 'p': 1.0, 'f': 0.21808510443993326}}
Reporting Criminal Incidents and Other Emergencies.txt
{'rouge-1': {'r': 0.22388059701492538, 'p': 1.0, 'f': 0.3658536555472933}, 'rouge-2': {'r': 0.15275813295615276, 'p': 0.972972972972973, 'f': 0.26405867736093763}, 'rouge-l': {'r': 0.22388059701492538, 'p': 1.0, 'f': 0.3658536555472933}}
Sexual Assault, Domestic Violence, Dating Violence.txt
{'rouge-1': {'r': 0.14438839848675913, 'p': 1.0, 'f': 0.2523415955911087}, 'rouge-2': {'r': 0.09474206349206349, 'p': 0.9646464646464646, 'f': 0.17253839042192698}, 'rouge-l': {'r': 0.14438839848675913, 'p': 1.0, 'f': 0.2523415955911087}}
CONFIDENTIAL REPORTING OF CRIME AND OTHER SERIOUS.txt
{'rouge-1': {'r': 0.09824561403508772, 'p': 1.0, 'f': 0.178913736390082

###LLM based Method

In [21]:
import os
from rouge import Rouge
from transformers import pipeline 

rouge = Rouge()
summarizer = pipeline("summarization")
path = "/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Data/Chapters/PreProcessed/"
os.chdir(path)

for file in os.listdir():
  if file.endswith(".txt"):
    file_path = f"{path}{file}"
    print(file)
    fileName = file.replace('.txt','')
    Lines = read_text_file(file_path)
    topic_content = listtostring(Lines)
    print(len(topic_content))
    max = int(len(topic_content)/10)
    if max > 1024:
      max = 1024
    summary = summarizer(topic_content, max_length=max, min_length=int(max*10/15), do_sample=False, truncation=True)
    
    print(rouge.get_scores(summary[0]['summary_text'], topic_content, avg=True))
    with open(r'/content/drive/MyDrive/CSCE Fall2022-Raxit/Quizzes/Quiz3/Results/Summarization/LLM/'+fileName+'.txt', 'w') as fp:
      fp.write(summary[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


The Division of Law Enforcement and Safety.txt
4257
{'rouge-1': {'r': 0.382089552238806, 'p': 0.9078014184397163, 'f': 0.5378151218809584}, 'rouge-2': {'r': 0.29422382671480146, 'p': 0.7951219512195122, 'f': 0.42951251252619}, 'rouge-l': {'r': 0.373134328358209, 'p': 0.8865248226950354, 'f': 0.5252100798641516}}
Reporting Criminal Incidents and Other Emergencies.txt
5373
{'rouge-1': {'r': 0.26119402985074625, 'p': 0.7142857142857143, 'f': 0.3825136572808982}, 'rouge-2': {'r': 0.18104667609618105, 'p': 0.5638766519823789, 'f': 0.2740899320807331}, 'rouge-l': {'r': 0.2562189054726368, 'p': 0.7006802721088435, 'f': 0.37522768278180896}}
Sexual Assault, Domestic Violence, Dating Violence.txt
36930
{'rouge-1': {'r': 0.09773013871374527, 'p': 0.8115183246073299, 'f': 0.17445132053494142}, 'rouge-2': {'r': 0.05084325396825397, 'p': 0.5338541666666666, 'f': 0.09284420131064905}, 'rouge-l': {'r': 0.09583858764186633, 'p': 0.7958115183246073, 'f': 0.17107484332616257}}
CONFIDENTIAL REPORTING OF 

##Document Similarity

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ["The right to be notified in writing of their rights in the conduct process;  The right to be assisted by various campus entities including but not limited to SAVIP, OSCAI, DLES;  The right to have an advisor of their choosing accompany them throughout the duration of the conduct process;  The right to submit a victim impact statement to the Hearing Officer or Council for consideration;  The right to have their past behaviors excluded in a University Conduct Hearing where responsibility is being de- termined;  The right to accommodations in giving testimony consistent with providing a safe atmosphere, and consistent with the rights of the accused;  The right to be notified in writing of the final determination and any sanction imposed on the accused as a result of the conduct process;  The right to receive a copy of the formal charges sent to the accused student;  The right to be notified of the date, time, and place of hearings at least three University business days prior to the hearing;  The right to have the hearing authority consider as an aggravating factor when sanctioning the perpetrator whether the perpetrator provided alcohol or other drugs in the commission of a sexual assault;  The right to be notified of the findings and sanctions/outcome of the hearing within a timeframe close to that in which the charged student was notified;  The right to appeal the outcome based on a due process error or on information that could not have been avail- able at the time of the hearing; and  The right to changes in academic, living, transportation, or working situations to avoid a hostile environment .", "the right to be treated with fairness, respect, and dignity, and to be free from intimidation, harassment, or abuse, throughout the criminal and juvenile justice process, and informed of the victim’s constitutional rights, provided by statute;  the right to be reasonably informed when the accused or convicted person is arrested, released from custody, or has escaped;  the right to be informed of and present at any criminal proceedings which are dispositive of the charges where the defendant has the right to be present;  the right to be reasonably informed of and be allowed to submit either a written or oral statement at all hearings affecting bond or bail;  the right to be heard at any proceeding involving a post-arrest release decision, a plea, or sentencing;  the right to be reasonably protected from the accused or persons acting on his behalf throughout the criminal justice process;  the right to confer with the prosecution, after the crime against the victim has been charged, before the trial or before any disposition and informed of the disposition;  the right to have reasonable access after the conclusion of the criminal investigation to all documents relating to the crime against the victim before trial;  the right to receive prompt and full restitution from the person or persons convicted of the criminal conduct that caused the victim’s loss or injury including both adult and juvenile offenders;  the right to be informed of any proceeding when any post-conviction action is being considered, and be present at any post-conviction hearing involving a post-conviction release decision;  the right to a reasonable disposition and prompt and final conclusion of the case; and  the right to have all rules governing criminal procedure and the admissibility of evidence in all criminal proceed- ings protect victims’ rights and have these rules subject to amendment or repeal by the legislature to ensure protec- tion of these rights Beyond these rights, DLES provides additional assistance to crime victims, including:  providing a free copy of incident reports when requested;  providing information about local victim assistance providers;  informing them of how to be compensated by the State Office of Victim Assistance (SOVA) and assistance in applying for compensation and other forms of assistance that may be available"]                                                                                                                                                                                                   
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
pairwise_similarity = tfidf * tfidf.T
pairwise_similarity.toarray()

array([[1.        , 0.34013856],
       [0.34013856, 1.        ]])

##REFERENCES
- https://github.com/biplav-s/course-nl/blob/master/l20-textsumm/ExploreExtractive.ipynb
- https://huggingface.co/sshleifer/distilbart-cnn-12-6
- https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis?text=I+like+you.+I+love+you
- https://nlp.stanford.edu/pubs/qi2020stanza.pdf
- https://huggingface.co/dslim/bert-large-NER?text=My+name+is+Sarah+and+I+live+in+London
- https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460